In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import time
from sklearn import metrics
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
path='dataset//reddit_train.csv'
path_test='dataset//reddit_test.csv'
labels=['Overwatch','Music','conspiracy','AskReddit','worldnews','anime','hockey','baseball','funny','nba',
    'canada','soccer','nfl','trees','gameofthrones','wow','leagueoflegends','GlobalOffensive','europe','movies']


In [2]:
data = pd.read_csv(path)
data_test = pd.read_csv(path_test)

In [3]:
X_train=data['comments']
y_train=data['subreddits']
X_test=data_test['comments']
print(X_test[0:5])

0    Trout and Bryant have both led the league in s...
1    &gt; Just like Estonians have good reasons to ...
2    Will Sol_Primeval sotp being oblivious?\n\nfin...
3    Moving Ostwald borders back to the pre 1967 bo...
4           You have to take it out of the bag, Morty!
Name: comments, dtype: object


In [4]:
vectorizer = CountVectorizer(stop_words='english')

vectors_train = vectorizer.fit_transform(X_train)
vectors_test = vectorizer.transform(X_test)
print(vectorizer)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [5]:
tf_idf_vectorizer = TfidfVectorizer(sublinear_tf=True)
vectors_train_idf = tf_idf_vectorizer.fit_transform(X_train)
vectors_test_idf = tf_idf_vectorizer.transform(X_test)
print(X_train.size)
print(X_test.size)

70000
30000


# Model Evaluation(Ensemble)

Voating Classifiers (8 models) imported from sklearn

In [20]:
from sklearn import datasets
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

#iris = datasets.load_iris()
#vectors_train_idf, y_train = iris.data, iris.target

clf1 = MultinomialNB(alpha=.2)
clf2 = LogisticRegression()
clf3 = RandomForestClassifier()
clf4 = DecisionTreeClassifier()
clf5 = BernoulliNB()
clf6 = LinearSVC(C=0.2)
clf7 = KNeighborsClassifier()
clf8 = SGDClassifier(average=True, alpha=0.00002, max_iter=20, tol=None)

eclf = VotingClassifier(estimators=[('MNB', clf1), ('LR', clf2),
                                    ('RFC', clf3), ('DTC', clf4), ('BNB', clf5), 
                                    ('SVC', clf6), ('KNN', clf7), ('SGD', clf8)], voting='hard')


for clf, label in zip([clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf8, eclf], ['MultinomialNB', 'LogisticRegression', 
                                                                         'RandomForestClassifier', 
                                                                         'DecisionTreeClassifier', 'BernoulliNB', 
                                                                         'LinearSVC', 'KNeighborsClassifier',
                                                                          'SGDClassifier', 'Ensemble']):
    start=time.time()
    scores = cross_val_score(clf, vectors_train_idf, y_train, cv=5, scoring='accuracy')
    end=time.time()
    print("Accuracy: %0.5f (+/- %0.5f) [%s]" % (scores.mean(), scores.std(), label))
    print("time(s): ",end-start)

Accuracy: 0.57083 (+/- 0.00323) [MultinomialNB]
time(s)):  2.5545105934143066


E:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: 0.54086 (+/- 0.00222) [LogisticRegression]
time(s)):  102.79083108901978


E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 

Accuracy: 0.31484 (+/- 0.00615) [RandomForestClassifier]
time(s)):  287.21384954452515
Accuracy: 0.26087 (+/- 0.00641) [DecisionTreeClassifier]
time(s)):  559.2987871170044
Accuracy: 0.47397 (+/- 0.00210) [BernoulliNB]
time(s)):  2.7424213886260986
Accuracy: 0.56733 (+/- 0.00200) [LinearSVC]
time(s)):  23.432509899139404
Accuracy: 0.06847 (+/- 0.00177) [KNeighborsClassifier]
time(s)):  277.2671570777893
Accuracy: 0.56627 (+/- 0.00141) [SGDClassifier]
time(s)):  35.95237350463867


E:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
E:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option

Accuracy: 0.57146 (+/- 0.00215) [Ensemble]
time(s)):  1336.9850368499756


Voating Classifiers (3 models) imported from sklearn

In [21]:
from sklearn import datasets
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier


clf1 = MultinomialNB(alpha=0.2)

clf2 = LinearSVC(C=0.2)
clf3 = SGDClassifier(average=True, alpha=0.00002, max_iter=20, tol=None)

eclf1 = VotingClassifier(estimators=[('MNB', clf1), 
                                    ('SVC', clf2),
                                     ('SGD', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf1], ['MultinomialNB', 
                                                 'LinearSVC', 
                                                  'SGDClassifier', 'Ensemble']):
    start=time.time()
    scores = cross_val_score(clf, vectors_train_idf, y_train, cv=5, scoring='accuracy')
    end=time.time()
    print("Accuracy: %0.5f (+/- %0.5f) [%s]" % (scores.mean(), scores.std(), label))
    print("time(s): ",end-start)

Accuracy: 0.57083 (+/- 0.00323) [MultinomialNB]
time(s):  3.421572685241699
Accuracy: 0.56733 (+/- 0.00200) [LinearSVC]
time(s):  33.591904163360596
Accuracy: 0.56604 (+/- 0.00176) [SGDClassifier]
time(s):  51.73287296295166
Accuracy: 0.57441 (+/- 0.00251) [Ensemble]
time(s):  81.51778721809387


Ensemble method 2 developed by ourselves

In [12]:
X_train, X_test, y_train, y_test = train_test_split(data['comments'], data['subreddits'], train_size=0.7, test_size=0.3)

tf_idf_vectorizer = TfidfVectorizer(sublinear_tf = True)
vectors_train_idf = tf_idf_vectorizer.fit_transform(X_train)
vectors_test_idf = tf_idf_vectorizer.transform(X_test)

In [13]:
#MultinomialNB
start=time.time()
from sklearn.naive_bayes import MultinomialNB
MNB_clf=MultinomialNB(alpha=0.2)
MNB_clf.fit(vectors_train_idf, y_train)
y_pred=MNB_clf.predict(vectors_test_idf)
mnb_report = metrics.classification_report(y_test,y_pred,output_dict=True)
print("test accuracy: %0.2f%%" %((y_pred==y_test).astype(np.int).mean()*100))

test accuracy: 56.96%


In [14]:
#SGDClassifier
from sklearn.linear_model import SGDClassifier
lsvm = SGDClassifier(average=True, alpha=0.00002, max_iter=100, tol=0.0001,loss="hinge")
lsvm.fit(vectors_train_idf, y_train)
y_pred_lsvm=lsvm.predict(vectors_test_idf)
lsvm_report = metrics.classification_report(y_test,y_pred_lsvm,output_dict=True)
print("test accuracy: %0.2f%%" %((y_pred_lsvm==y_test).astype(np.int).mean()*100))

test accuracy: 56.13%


In [15]:
#LinearSVC
from sklearn.svm import LinearSVC
svc_clf = LinearSVC(C=0.2)
svc_clf.fit(vectors_train_idf, y_train)
y_pred_svc=svc_clf.predict(vectors_test_idf)
svc_report = metrics.classification_report(y_test,y_pred_svc,output_dict=True)
print("test accuracy: %0.2f%%" %((y_pred_svc==y_test).astype(np.int).mean()*100))

test accuracy: 56.50%


In [16]:
def developed_ensemble(label_a,label_b,label_d):
    if label_b==label_d:
        a = mnb_report[label_a]['precision']
        b = svc_report[label_b]['precision']
        d = lsvm_report[label_d]['precision']
        if a == max(a,b,d):
            return label_a
        elif b == max(a,b,d):
            return label_b
        else:
            return label_d
    else:
        return label_a

In [17]:
y_pred_final = np.copy(y_pred)
for i in range(len(y_pred)):
    y_pred_final[i] = developed_ensemble(y_pred[i], y_pred_svc[i],y_pred_lsvm[i])
    
print("test accuracy: %0.2f%%" %((y_pred_final==y_test).astype(np.int).mean()*100))
end=time.time()
print("time:",end-start)

test accuracy: 57.71%
time: 14.410178184509277
